<a href="https://colab.research.google.com/github/SANJAY729/Decision_Tree/blob/master/allo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import copy

In [34]:
def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy


def InfoGain(data,split_attribute_name,target_name="class"):
    total_entropy = entropy(data[target_name])
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain


def ID3(data,originaldata,features,max_depth,target_attribute_name="class",parent_node_class = None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    elif len(data) == 0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    elif len(features) == 0 or max_depth == 0:
        return parent_node_class
    
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] 
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        tree = {best_feature:{},"recurrence-events":0,"no-recurrence-events":0}
        features = [i for i in features if i != best_feature]
        for value in np.unique(data[best_feature]):
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data,dataset,features,max_depth-1,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree
        return(tree)    


def predict(query,tree):
    default = "no-recurrence-events"
    if (tree["recurrence-events"]>tree["no-recurrence-events"]):
        default = "recurrence-events"
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result

def test(data,tree):
    queries = data.iloc[:,1:].to_dict(orient = "records")
    predicted = pd.DataFrame(columns=["predicted"]) 
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree)
    return (np.sum(predicted["predicted"] == data["class"])/len(data))*100
    
def majorityclassutil(query,tree):
    if (query["class"]=="no-recurrence-events"):
        tree["no-recurrence-events"] = tree["no-recurrence-events"] + 1
    else:
        tree["recurrence-events"] = tree["recurrence-events"] + 1
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return majorityclassutil(query,result)
            else:
                return

def majorityclasses(data,tree):
     queries = data.iloc[:,:-1].to_dict(orient = "records")
     for i in range(len(data)):
        majorityclassutil(queries[i],tree)






In [35]:
dataset_init = pd.read_csv('breast-cancer.csv',
                      names=['class','age','menopause','tumor-size','inv-nodes','node-caps','def-malig','breast','breast-quad','irradiat'])
# 1. class: no-recurrence-events, recurrence-events
# 2. age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.
# 3. menopause: lt40, ge40, premeno.
# 4. tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59.
# 5. inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39.
# 6. node-caps: yes, no.
# 7. deg-malig: 1, 2, 3.
# 8. breast: left, right.
# 9. breast-quad: left-up, left-low, right-up, right-low, central.
# 10. irradiat: yes, no.

dataset_ = dataset_init[dataset_init['node-caps']!='?']
dataset = dataset_[dataset_['breast-quad']!='?']


In [36]:
def train_validation_test_split(dataset):
    training_data = dataset.iloc[:180].reset_index(drop=True)
    validation_data = dataset.iloc[180:230].reset_index(drop=True)
    testing_data = dataset.iloc[230:].reset_index(drop=True)
    return training_data,validation_data,testing_data

In [37]:
#QUESTION 1 
###################################################
max_depth = int(input("Enter the maximum depth : "))
accuracy = 0
for i in range (1,11):
    shuffled_data = copy.deepcopy(dataset[1:])
    shuffled_data = shuffled_data.sample(frac=1, random_state = i).reset_index(drop=True)
    training_data,validation_data,testing_data = train_validation_test_split(shuffled_data)
    tree = ID3(training_data,training_data,training_data.columns[1:],i)
    majorityclasses(training_data,tree)
    accuracy = accuracy + test(testing_data,tree)
print("The average accuracy over 10 different splits is ",accuracy/10)

Enter the maximum depth : 5
The average accuracy over 10 different splits is  69.78260869565217


<function matplotlib.pyplot.show>

In [38]:
x = [1,2,3,4,5,6,7,8,9]
y = []
for i in range (1,10):
    accuracy = 0
    for j in range(1,11):
        shuffled_data = copy.deepcopy(dataset[1:])
        shuffled_data = shuffled_data.sample(frac=1, random_state = i).reset_index(drop=True)
        training_data,validation_data,testing_data=train_validation_test_split(shuffled_data)
        tree = ID3(training_data,training_data,training_data.columns[1:],i)
        majorityclasses(training_data,tree)
        accuracy = accuracy + test(testing_data,tree)
    y.append(accuracy/10)
plt.plot(x,y)
plt.ylabel('Accuracy')
plt.xlabel('Max Depth')
plt.show


In [39]:
def possiblevalues(keys):
    feature = ""
    values = []
    for key in keys:
        if (key == 'no-recurrence-events'):
            continue
        elif (key == 'recurrence-events'):
            continue
        else :
            feature = key
            if (feature == 'age'):
                values = ['10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99']
            if (feature == 'menopause'):
                values = ['lt40', 'ge40', 'premeno']
            if (feature == 'tumor-size'):
                values = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59']
            if (feature == 'inv-nodes'):
                values = ['0-2', '3-5', '6-8', '9-11', '12-14', '15-17', '18-20', '21-23', '24-26', '27-29', '30-32', '33-35', '36-39']
            if (feature == 'node-caps'):
                values = ['yes', 'no']
            if (feature == 'deg-malig'):
                values = ['1', '2', '3']
            if (feature == 'breast'):
                values = ['left', 'right']
            if (feature == 'breast-quad'):
                values = ['left-up', 'left-low', 'right-up', 'right-low', 'central']
            if (feature == 'irradiat'):
                values = ['yes', 'no']            
    return feature,values

def skip():
  return
def delete_node(tree,node,target):
    if (node==tree):
        return target
    else:
        if isinstance(tree, dict):
            feature, values = possiblevalues(tree.keys())
            for value in values:
                try:
                    tree[feature][value] = copy.deepcopy(delete_node(tree[feature][value],node,target))
                except:
                    skip()
    return tree
def prune_singlenode(tree,initial_accuracy,validation_data):
    initial_tree = copy.deepcopy(tree)
    best_tree = copy.deepcopy(tree)
    current_tree = copy.deepcopy(tree)
    best_accuracy = initial_accuracy
    flag = 0
    queue = []
    node_removed=None
    queue.append(tree)
    while queue!=[]:
        node = queue.pop(0)
        if isinstance(node, dict):
            feature, values = possiblevalues(node.keys())
            for value in values:
                try:
                    queue.append(node[feature][value])
                    
                except:
                    skip()
            if node==initial_tree :
                continue
            
            if node['no-recurrence-events']>node['recurrence-events']:
                current_tree=copy.deepcopy(delete_node(initial_tree,node,'no-recurrence-events'))
            else :
                current_tree=copy.deepcopy(delete_node(initial_tree,node,'recurrence-events'))
            initial_tree = copy.deepcopy(tree)
            #print(current_tree)
            current_accuracy = test(validation_data,current_tree)
            if current_accuracy>best_accuracy:
                flag = 1
                best_tree = copy.deepcopy(current_tree)
                node_removed = node
                best_accuracy = current_accuracy    
    return best_tree,best_accuracy,flag

In [40]:
max_depth  = 3 #from the graph
dataset = dataset.sample(frac=1).reset_index(drop=True)
training_data,validation_data,testing_data = train_validation_test_split(dataset)
tree = ID3(training_data,training_data,training_data.columns[1:],max_depth)
majorityclasses(training_data,tree)

initial_accuracy_validation = test(validation_data,tree)
initial_accuracy_test = test(testing_data,tree)

ini_tree=copy.deepcopy(tree)

new_tree,new_accuracy,flag = prune_singlenode(tree,initial_accuracy_validation,validation_data)
while (flag==1):
    new_tree,new_accuracy,flag = prune_singlenode(new_tree,new_accuracy,validation_data)

final_accuracy_validation = test(validation_data,new_tree)
final_accuracy_test = test(testing_data,new_tree)
print(initial_accuracy_validation)
print(initial_accuracy_test)
print(final_accuracy_validation)
print(final_accuracy_test)

hello
hello
hello
hello
68.0
63.829787234042556
74.0
61.702127659574465
